In [60]:
#DONT RUN. TAKES A LOT OF TIME TO RUN\

In [59]:
import joblib
import numpy as np
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx
import heapq

In [44]:
def get_data(file):
    return joblib.load(file)

In [45]:
def tokenize(df):
    sentences = []
    for s in df['review']:
      sentences.append(sent_tokenize(s))

    sentences = [y for x in sentences for y in x] 
    return sentences

In [46]:
def get_word_embeddings():
    word_embeddings = {}
    f = open('glove.6B.100d.txt', encoding='utf-8')
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        word_embeddings[word] = coefs
    f.close()
    return word_embeddings

In [47]:
def preprocess(sentences):
    def remove_stopwords(sen):
        sen_new = " ".join([i for i in sen if i not in stop_words])
        return sen_new
    clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")
    clean_sentences = [s.lower() for s in clean_sentences]
    stop_words = stopwords.words('english')
    clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]
    return clean_sentences

In [48]:
def get_sentences_vector(clean_sentences, word_embeddings):
    sentence_vectors = []
    for i in clean_sentences:
        if len(i) != 0:
            v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
        else:
            v = np.zeros((100,))
        sentence_vectors.append(v)
    return sentence_vectors

In [49]:
def get_similarity_matrix(sentences, sentence_vectors):
    sim_mat = np.zeros([len(sentences), len(sentences)])
    for i in range(len(sentences)):
        if(i%10) == 0:
            print(i)
        for j in range(len(sentences)):
            if i != j:
              sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]
    return sim_mat

In [50]:
#GET DATA
df = get_data("data/dataframe_nearest_1day.pkl")
df.rename(columns={'text_x':'tip', 'text_y':'review'}, inplace=True)
df = df[:100]
sentences = tokenize(df)
word_embeddings = get_word_embeddings()
print(len(sentences))
sentences = preprocess(sentences)
sentences_vector = get_sentences_vector(sentences, word_embeddings)
similarity_matrix = get_similarity_matrix(sentences, sentences_vector)

618


In [56]:
nx_graph = nx.from_numpy_array(similarity_matrix)
scores = nx.pagerank(nx_graph)

In [57]:
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)